In [1]:
import numpy as np

from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    Simulator,
    StartMeshcat,
)
from pydrake.common import temp_directory
from pydrake.examples import ManipulationStation
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph

In [2]:
 # Start meshcat server
meshcat = StartMeshcat()
playback = True
duration = 1.0
# model = "pend"
model = "manip"

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
def run_pendulum_example():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
    parser = Parser(plant)
    parser.AddModelsFromUrl(url="package://drake/examples/pendulum/Pendulum.urdf")
    plant.Finalize()

    # Add Meshcat visualizer
    meshcat_viz = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    if playback:
        meshcat_viz.StartRecording()

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.0)

    # Fix the input port to zero.
    plant_context = diagram.GetMutableSubsystemContext(
        plant, simulator.get_mutable_context()
    )
    plant.get_actuation_input_port().FixValue(
        plant_context, np.zeros(plant.num_actuators())
    )
    plant_context.SetContinuousState(np.array([0.5, 0.1]))
    simulator.AdvanceTo(duration)

    if playback:
        meshcat_viz.StopRecording()
        meshcat_viz.PublishRecording()


def run_manipulation_example():
    # Start meshcat server
    meshcat = StartMeshcat()

    builder = DiagramBuilder()
    station = builder.AddSystem(ManipulationStation(time_step=0.005))
    station.SetupClutterClearingStation()
    station.Finalize()

    scene_graph = station.get_scene_graph()

    # Add Meshcat visualizer
    meshcat_viz = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    if playback:
        meshcat_viz.StartRecording()

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.0)

    # Fix the control inputs to zero.
    station_context = diagram.GetMutableSubsystemContext(
        station, simulator.get_mutable_context()
    )
    station.GetInputPort("iiwa_position").FixValue(
        station_context, station.GetIiwaPosition(station_context)
    )
    station.GetInputPort("iiwa_feedforward_torque").FixValue(
        station_context, np.zeros(7)
    )
    station.GetInputPort("wsg_position").FixValue(station_context, np.zeros(1))
    station.GetInputPort("wsg_force_limit").FixValue(station_context, np.array([40.0]))
    simulator.AdvanceTo(duration)

    if playback:
        meshcat_viz.StopRecording()
        meshcat_viz.PublishRecording()


def main():
    np.set_printoptions(precision=5, suppress=True)
    if model == "pend":
        run_pendulum_example()
    elif model == "manip":
        run_manipulation_example()

In [ ]:
main()

INFO:drake:Meshcat listening for connections at http://localhost:7002


INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/b1e6a452495c59eec34f849e76cc78f7631473c4.tar.gz


RuntimeError: DiagramBuilder: System 'scene_graph' has not been registered to this DiagramBuilder using AddSystem nor AddNamedSystem.

The systems currently registered to this builder are: 'manipulation_station', 'meshcat_visualizer(visualizer)'.

If 'scene_graph' was registered as a subsystem to one of these, you must export the input or output port using ExportInput/ExportOutput and then connect to the exported port.